# FEM from scratch: Laplace problem

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

We implement the $\mathbb{P}^1$ FEM for solving the Laplace problem with Dirichlet and Neumann boundary conditions. On the unit square $\Omega = {[0,1]}^2$ we consider 
\begin{equation*}
  \begin{cases}
    -\Delta u = f, \qquad & \text{in }\Omega, \\
    u = g_D, \qquad & \text{on }\Gamma_D, \\
    \boldsymbol{\nabla}{u}\cdot\boldsymbol{n} = g_N & \text{on }\Gamma_N.
  \end{cases}
\end{equation*}

We test the solver using the simple analytical solution given by 
$$
u(x,y) = (1-x^2)(1-y^2),
$$
from which we infer $f = 4 - 2(x^2+y^2)$. .
We consider three different configurations of boundary conditions:

1.   **Problem 0**: full Dirichlet conditions, i.e.  $\Gamma_D = \partial\Omega$
2.   **Problem 1**: mixed homogeneous conditions, i.e. $\Gamma_D := \{ (x,y)\in\partial\Omega\ | x=1 \} \cup \{ (x,y)\in\partial\Omega\ | y=1 \}$ and $\Gamma_N :=\partial\Omega\setminus\Gamma_D$ (which, indeed, yields $g_D=0$ and $g_N=0$). 
3.   **Problem 2**: mixed non-homogeneous conditions, i.e.
$\Gamma_D := \{ (x,y)\in\partial\Omega\ | x=0 \} \cup \{ (x,y)\in\partial\Omega\ | x=1 \}$ and $\Gamma_N :=\partial\Omega\setminus\Gamma_D$. 



### Step 1: Mesh generation and reordering of boundary nodes

In [ ]:
def left_mesh(a, b, nx, ny):
  '''
  construct a left-type uniform triangular mesh of [0,a] x [0,b] starting from a 
  cartesian grid with nx cells in the x-direction and ny cells in the y-direction
  '''
  P = np.zeros(((nx+1)*(ny+1),2))
  T = np.zeros((2*nx*ny,3), dtype=int)

  x = np.linspace(0, a, nx+1)
  y = np.linspace(0, b, ny+1)

  for i in range(0, nx+1):
      for j in range(0, ny+1):
          # create coordinate matrix P (ordered from bottom-left to top-right)
          P[i*(ny+1)+j,0] = x[i]
          P[i*(ny+1)+j,1] = y[j]

          # create connectivity matrix T with local backward orientation
          if (i<nx) and (j<ny):
              T[2*(i*ny+j),0] = i*(ny+1) + j
              T[2*(i*ny+j),1] = (i+1)*(ny+1) + j
              T[2*(i*ny+j),2] = i*(ny+1) + j+1
              
              T[2*(i*ny+j)+1,0] = i*(ny+1) + j+1
              T[2*(i*ny+j)+1,1] = (i+1)*(ny+1) + j
              T[2*(i*ny+j)+1,2] = (i+1)*(ny+1) + j+1

  return P, T

In [ ]:
def Dirichlet(a, b, Point, tol, pb = 0):
  '''
  condition defining the Dirichlet boundary
  '''
  edge_0 = Point[0] < tol
  edge_1 = Point[1] < tol 
  edge_2 = Point[0] > a-tol
  edge_3 = Point[1] > b-tol
  
  if pb == 0:    # full Dirichlet problem
    diric_bc = (edge_0 or edge_1) or (edge_2 or edge_3)
  elif pb == 1:  # mixed homogeneous problem
    diric_bc = edge_2 or edge_3
  elif pb == 2:  # mixed non-homogeneous problem
    diric_bc = edge_0 or edge_2
  else:
    print('wrong pb argument:') 
    print('  0= full Dirichelt, 1= mixed homogeneous, 2= mixed non-homogeneous')

  return diric_bc

In [ ]:
def Neumann(a, b, Point, tol, pb = 0):
  '''
  condition defining the Neumann boundary
  '''
  edge_0 = Point[0] < tol
  edge_1 = Point[1] < tol 
  edge_2 = Point[0] > a-tol
  edge_3 = Point[1] > b-tol

  if pb == 0:    # full Dirichlet problem
    neuma_bc = False
  elif pb == 1:  # mixed homogeneous problem
    neuma_bc = edge_0 or edge_1
  elif pb == 2:  # mixed non-homogeneous problem
    neuma_bc = edge_1 or edge_3
  else:
    print('wrong pb argument:') 
    print('  0= full Dirichelt, 1= mixed homogeneous, 2= mixed non-homogeneous')

  return neuma_bc

In [ ]:
def ordered_mesh(a, b, nx, ny, pb = 0):
  '''
  construct a left-type uniform triangular mesh of [0,a] x [0,b] starting from a 
  cartesian grid with nx cells in the x-direction and ny cells in the y-direction
  P,T are modified in order to put internal nodes first and Dirichlet nodes last
  '''
  P, T = left_mesh(a, b, nx, ny)
  P_new = np.zeros(P.shape)
  T_new = np.zeros(T.shape, dtype = int)
  
  n_bndry = 2 * (nx + ny) 
  n_inter = 0
  n_diric = 0
  n_neumn = 0

  tol = min(a/(2*nx), b/(2*ny))
  v = np.zeros(P[:,0].size, dtype = int)     # permutation map

  for i in range(P[:,0].size):

    if Dirichlet(a, b, P[i,:], tol, pb):
      # node i belongs to Dirichlet boundary
      v[i] = P[:,0].size - n_diric - 1
      n_diric += 1
    elif Neumann(a, b, P[i,:], tol, pb):
      # node i belongs to Neumann boundary
      v[i] = P[:,0].size - n_bndry + n_neumn 
      n_neumn += 1
    else: 
      # node i is internal 
      v[i] = n_inter
      n_inter += 1
    
    # update nodes matrix P 
    P_new[v[i],:] = P[i,:]

  # update elements matrix T
  T_new[:,:] = v[T[:,:]]

  return P_new, T_new, n_diric, n_neumn

In [ ]:
def plot_mesh(P, E):
  for i in range(E[:,0].size):
    x = [P[E[i, E[0,:].size-1], 0], P[E[i, 0], 0]]
    y = [P[E[i, E[0,:].size-1], 1], P[E[i, 0], 1]]
    plt.plot(x, y, "k")
    for j in range(E[0,:].size-1):
      x = [P[E[i, j], 0], P[E[i, j+1], 0]]
      y = [P[E[i, j], 1], P[E[i, j+1], 1]]
      plt.plot(x, y, "k")
        
  plt.show()

In [ ]:
# ordered_mesh(a, b, nx, ny, pb = 0)
P, T, n_diric, n_neumn = ordered_mesh(1, 1, 4, 2, 0)
plot_mesh(P,T)
print(P)

### Assembling of stiffness matrix and right-hand side

Let $\phi_i$ be the hat-function associated to the $i$-th node in the mesh $\mathcal{T}_h$. We denote with $\mathcal{E}_h^N$ the set of boundary edges belonging to the Neumann boundary. The linear system associated to the FEM approximation reads $A \boldsymbol{u} = \boldsymbol{f}$, where

*   The stiffness matrix $A$ is given by:
$$
A_{ij} = \sum_{T\in \mathcal{T}_h}\int_T \boldsymbol{\nabla} \varphi_i \cdot \boldsymbol{\nabla} \varphi_j
$$
*   the right-hand side vector $\boldsymbol{f}$ is given by:
$$
f_i = \sum_{T\in \mathcal{T}_h}\int_T f\varphi_i +
\sum_{E\in \mathcal{E}_h^N}\int_E g_N\varphi_i 
$$

In **problem 1** and **problem 2** the contribution to $\boldsymbol{F}$ associated to the Neumann BC vanishes. 
Moreover, we can deal with non-homogenoeous Dirichlet conditions using the usual trick. Thus, we start by focusing on the assembling of $A$ using the reference element approach and the term corresponding to the volumetric load function. 

The term associated to the load function $\boldsymbol{f}$ is computed using a simple quadrature rule (evaluation at the center of each element).

In [ ]:
def assemble(P, T, f):
  '''
  construct the Grad*Grad stiffness matrix and rhs vector 
  corresponding to mesh (P,T) and load function f
  '''
  n_nodes = P[:,0].size
  n_elems = T[:,0].size
  A = np.zeros((n_nodes, n_nodes))
  b = np.zeros(n_nodes)

  # gradients of basis functions on the reference element
  grad_ref = np.array(((-1,  1,  0),
                       (-1,  0,  1)))

  for k in range(n_elems):
    # compute edges of element k
    B = np.zeros((2,2))
    B[:, 0] = (P[T[k, 1], :] - P[T[k, 0], :]).T #subtract two rows of the coordinate matrix
    B[:, 1] = (P[T[k, 2], :] - P[T[k, 0], :]).T
    # the determinant of B is the area of a parallelogram
    # the area of the triangle is half of that
    J = np.linalg.det(B)
    area_T = J/2.0
    invB = np.linalg.inv(B)
    A_loc = grad_ref.T @ invB @ invB.T @ grad_ref * J/2
    

    # find the barycenter
    bar_T = np.array((P[T[k,0],:] + P[T[k,1],:] + P[T[k,2],:])/3.0)
 
    # each local contribution is stored in the corresponding global position
    for iloc in range(3):
      # the value of each shape function at the barycenter is 1/3
      b[T[k,iloc]] += area_T * f(bar_T[0], bar_T[1]) / 3.0
      for jloc in range(3):
        A[T[k,iloc], T[k,jloc]] += A_loc[iloc, jloc]

  return A, b

In [ ]:
def neumann_rhs(a, b, Point, grad_u, tol):
  '''
  returns the average flux in the two Neumann edges sharing 'Point' 
  '''
  # on the domain boundary we define the outward-pointing normal vectors
  # corresponding to the two edges sharing 'Point' 
  if (Point[0] < tol) and (Point[1] < b-tol):
     normal_l = np.array([-1, 0])
     if Point[1] < tol:
       normal_r = np.array([0, -1])
     else:
       normal_r = normal_l
  elif Point[1] < tol:  
     normal_l = np.array([0, -1])
     if Point[0] > a-tol:
       normal_r = np.array([1, 0])
     else:  
       normal_r = normal_l
  elif Point[0] > a-tol: 
     normal_l = np.array([1, 0])
     if Point[1] > b-tol:
       normal_r = np.array([0, 1])
     else:
       normal_r = normal_l
  else:
     normal_l = np.array([0, 1])
     if Point[0] < tol:
       normal_r = np.array([-1, 0])
     else:
       normal_r = normal_l
  
  tangent_l = np.array([normal_l[1], -normal_l[0]]) * tol 
  tangent_r = np.array([-normal_r[1], normal_r[0]]) * tol 
  flux_l = np.dot(grad_u((Point+tangent_l)[0], (Point+tangent_l)[1]), normal_l)
  flux_r = np.dot(grad_u((Point+tangent_r)[0], (Point-tangent_r)[1]), normal_r)
  flux = tol*(flux_l + flux_r)

  return flux

### Main program

In [ ]:
pb = 2
n = 20
f = lambda x, y: 4 - 2* (x**2+y**2)
u = lambda x, y: (1-x**2)*(1-y**2)
def grad_u(x,y): 
  grad = np.array([-2*x*(1-y**2), -2*y*(1-x**2)])
  return grad

# Step 1: meshing
P, T, n_diric, n_neumn = ordered_mesh(1, 1, n, n, pb)

# Step 2: definitions of discrete space (dofs)
n_nodes = P[:,0].size
n_dofs = n_nodes - n_diric
uh = np.zeros(n_nodes)

# Step 3: assembling matrices and rhs vectors
A, b = assemble(P, T, f)
# accounting for Neumann BCs
for j in range(n_dofs-n_neumn, n_dofs):
  b[j] += neumann_rhs(1, 1, P[j,:], grad_u, 1/(2*n))
# Enforce Dirichlet BCs
A[n_dofs:n_nodes, :] = 0  # rows corresponding to Dirichlet nodes are set to zero
for j in range(n_dofs, n_nodes):
    A[j, j] = 1 # and then the diagonal is set to one
    b[j] = u(P[j,0], P[j,1]) #set the rhs to the Dirichlet condition

# Step 4: solving linear system
uh = np.linalg.solve(A, b)


# Step 5a: compute errors
err_h1 = 0
err_l2 = 0
for k in range(T[:,0].size):
  # compute edges of element k
  e = np.zeros((3,2))
  e[2,:] = P[T[k,1],:] - P[T[k,0],:]
  e[0,:] = P[T[k,2],:] - P[T[k,1],:]
  e[1,:] = P[T[k,0],:] - P[T[k,2],:] 
  area_T = np.linalg.det(e[0:2,:])/2.0
  bar_T = np.array((P[T[k,0],:] + P[T[k,1],:] + P[T[k,2],:])/3.0)

  gradu_bar = grad_u(bar_T[0], bar_T[1])
  grad_uh = np.array([sum(-e[:,1]*uh[T[k,:]]), sum(e[:,0]*uh[T[k,:]])])/(2*area_T)
  err_h1 += area_T * np.dot(gradu_bar - grad_uh, gradu_bar - grad_uh)

  u_bar = u(bar_T[0], bar_T[1])
  uh_bar = (uh[T[k,0]] + uh[T[k,1]] + uh[T[k,2]])/3.0
  err_l2 += area_T * (uh_bar - u_bar)**2

err_h1 = np.sqrt(err_h1)
err_l2 = np.sqrt(err_l2)
print('n={} eL2={:.2e} eH1={:.2e}'.format(n, err_l2, err_h1))

# Step 5b: plot the results and compare with true solution
Q, T = left_mesh(1,1,4*n,4*n)
u_ex = u(Q[:,0], Q[:,1])
fig, axes = plt.subplots(1,2, figsize=(12,6))

h0 = axes[0].tricontourf(P[:,0], P[:,1], uh , 20)
axes[0].set_title("FEM approximation")
plt.colorbar(h0, ax=axes[0])

h1 = axes[1].tricontourf(Q[:,0], Q[:,1], u_ex , 20)
axes[1].set_title("exact solution")
plt.colorbar(h1, ax=axes[1])

fig.tight_layout()
plt.show()

An alternative assemble function is reported below. It is based on the following considerations:
let the nodes $P_i$ and $P_j$ belong to the boundary of element $T$ and let $e_i$ and $e_j$ be the edges of $T$ opposed to the vertices $P_i$ and $P_j$, respectively. We observe that 
$$
(\boldsymbol{\nabla}\varphi_i)_{|T} = \frac{R_{\pi/2} \boldsymbol{e}_i}{2\ |T|}, \quad\text{ and }\;\;
(\boldsymbol{\nabla}\varphi_i\cdot\boldsymbol{\nabla}\varphi_j)_{|T} =
\frac{\boldsymbol{e}_i \cdot \boldsymbol{e}_j}{4\ |T|^2}.
$$
where $R_{\pi/2}$ denotes a rotation matrix.
Thus, $\int_T \boldsymbol{\nabla}\varphi_i\cdot\boldsymbol{\nabla}\varphi_j = \frac{\boldsymbol{e}_i \cdot \boldsymbol{e}_j}{4\ |T|} $.
 

In [ ]:
def assemble_alt(P, T, f):
  '''
  construct the Grad*Grad stiffness matrix and rhs vector 
  corresponding to mesh (P,T) and load function f
  '''
  n_nodes = P[:,0].size
  n_elems = T[:,0].size
  M = np.zeros((n_nodes, n_nodes))
  b = np.zeros(n_nodes)

  for k in range(n_elems):
    # compute edges of element k
    e = np.zeros((3,2))
    e[2,:] = P[T[k,1],:] - P[T[k,0],:]
    e[0,:] = P[T[k,2],:] - P[T[k,1],:]
    e[1,:] = P[T[k,0],:] - P[T[k,2],:] 
    area_T = np.linalg.det(e[0:2,:])/2.0
    # find the barycenter
    bar_T = np.array((P[T[k,0],:] + P[T[k,1],:] + P[T[k,2],:])/3.0)
 
    # each local contribution is stored in the corresponding global position
    for iloc in range(3):
      b[T[k,iloc]] += area_T * f(bar_T[0], bar_T[1]) / 3.0;
      for jloc in range(3):
        M[T[k,iloc], T[k,jloc]] += np.dot(e[iloc,:], e[jloc,:])/(4*area_T)

  return M, b